## Switches and flags

In [ ]:
ExtractIndividualStudies = True
ExtractGroupedStudies = True
AggregateStorageLevels = True
NumSrVars = 66
ConvertAcFtToTaf = True

## Import standard libraries

In [ ]:
import os
import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')
import datetime as dt
import pandas as pd
import numpy as np
import cqwlutils as cu

## Import custom modules

In [ ]:
# Import custom modules - NEED WINDOWS OS (NOTE: I had to run this twice, must check why this happens!)
from coeqwalpackage.DataExtraction import *

## Define contol file name

In [ ]:
CtrlFile = 'CalSim3GroundWaterDataExtractionInitFile_v1.xlsx'
CtrlTab = 'Init'

## Read from control file

In [ ]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, GW1DssNamesOutPath, GW2DssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, GW1DssMin, GW1DssMax, GW2DssMin, GW2DssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

In [ ]:
print([ScenarioListFile, ScenarioListTab, ScenarioListPath, GW1DssNamesOutPath, GW2DssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, GW1DssMin, GW1DssMax, GW2DssMin, GW2DssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax])


## Check for output directory and create if necessary

In [ ]:
# check if output directory exists
if not os.path.exists(GroupDataDirPath):
    # print warning
    print("Warning: directory " + GroupDataDirPath + " does not exists and will be created")
    
    # Create the directory
    os.makedirs(GroupDataDirPath)


## Define Nan Values

In [ ]:
# NaN values as defined by CalSim3
Nan1 = -901
Nan2 = -902

## Read indeces, dss names, directory names, start and end dates, time range

In [ ]:
gw1dsshdr, gw1dssname = cu.read_from_excel(ScenarioListPath, ScenarioListTab, GW1DssMin, GW1DssMax, hdr=True)
gw1dss_names = []
for i in range(len(gw1dssname)):
    gw1dss_names.append(gw1dssname[i][0])
gw1dss_names

In [ ]:
gw2dsshdr, gw2dssname = cu.read_from_excel(ScenarioListPath, ScenarioListTab, GW2DssMin, GW2DssMax, hdr=True)
gw2dss_names = []
for i in range(len(gw2dssname)):
    gw2dss_names.append(gw2dssname[i][0])
gw2dss_names

In [ ]:
indexhdr, index_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, IndexMin, IndexMax, hdr=True)
index_names = []
for i in range(len(index_name)):
    index_names.append(index_name[i][0])
index_names

In [ ]:
studyhdr, study_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, NameMin, NameMax, hdr=True)
study_names = []
for i in range(len(study_name)):
    study_names.append(study_name[i][0])
study_names

In [ ]:
dirhdr, dir_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, DirMin, DirMax, hdr=True)
dir_names = []
for i in range(len(dir_name)):
    dir_names.append(dir_name[i][0])
dir_names

In [ ]:
starthdr, start_date = cu.read_from_excel(ScenarioListPath, ScenarioListTab, StartMin, StartMax, hdr=True)
start_dates = []
for i in range(len(start_date)):
    start_dates.append(start_date[i][0])
datetime_start_dates = pd.to_datetime(start_dates)
# turns out that dss reading library wands a dt datetime, not pd datetime
dt_datetime_start_dates = [dt.to_pydatetime() for dt in datetime_start_dates]


In [ ]:
endhdr, end_date = cu.read_from_excel(ScenarioListPath, ScenarioListTab, EndMin, EndMax, hdr=True)
end_dates = []
for i in range(len(end_date)):
    end_dates.append(end_date[i][0])
# turns out that dss reading library wands a dt datetime, not pd datetime
datetime_end_dates = pd.to_datetime(end_dates)
dt_datetime_end_dates = [dt.to_pydatetime() for dt in datetime_end_dates]


In [ ]:
min_datetime = min(dt_datetime_start_dates)
print('Min time: ')
print(min_datetime)
max_datetime = max(dt_datetime_end_dates)
print('Max time: ')
print(max_datetime)


## Set path and file names for indeces, dss names and directory names and write files

In [ ]:
gw1dss_df = (pd.DataFrame(gw1dss_names))
gw1dss_df.to_csv(GW1DssNamesOutPath)
print(GW1DssNamesOutPath)
index_df = (pd.DataFrame(gw1dss_names))
index_df.to_csv(ScenarioIndicesOutPath)
print(ScenarioIndicesOutPath)
dir_df = (pd.DataFrame(gw1dss_names))
dir_df.to_csv(DssDirsOutPath)
print(DssDirsOutPath)


## Read and write variables list

In [ ]:
# get vars
hdr, vars = cu.read_from_excel(VarListPath, VarListTab,VarMin,VarMax,hdr=True)
gw1var_df = pd.DataFrame(data=vars, columns=hdr)
gw1var_df

In [ ]:
# write vars
gw1var_df.to_csv(VarOutPath)
VarOutPath

## Loop on DSS files, extract variables and write to scenario-specific CSVs

In [ ]:
if ExtractIndividualStudies:
    for i in range(len(gw1dss_names)):
        gw1dss_name = os.path.join(ScenarioDir, gw1dss_names[i])
        datetime_start_date = dt_datetime_start_dates[i]
        datetime_end_date = dt_datetime_end_dates[i]
        print('\nReading ' + gw1dss_name + '\n')   
        df = preprocess_GW_data_study_dss(gw1var_df, gw1dss_name, datetime_start_date, datetime_end_date, addSRlevels = AggregateStorageLevels, num_vars = NumSrVars, convertAcFtToTaf = ConvertAcFtToTaf)
        file_path = gw1dss_name.replace(ModelSubPath, ExtractionSubPath).replace(".dss", ".csv")
        dir_path = os.path.dirname(file_path)
        os.makedirs(dir_path, exist_ok=True)
        print(df.tail(5))
        print('\nWriting ' + file_path + '\n')
        df.to_csv(file_path, na_rep="NaN", header=True, index=True)

## Read all DSS files, combine variables and write to single CSV
#### Note: warnings will appear whenever a study length is shorter than the max time range

In [ ]:
if ExtractGroupedStudies:
    df = preprocess_compound_GW_data_dss(gw1var_df, ScenarioDir, gw1dss_names, index_names, min_datetime, max_datetime, addSRlevels = AggregateStorageLevels, num_vars = NumSrVars)
    df.replace([Nan1, Nan2], np.nan, inplace=True)

In [ ]:
if ExtractGroupedStudies:
    print(df)

## Write the compund dataframe to CSV

In [ ]:
# write the dataframe to CSV
if ExtractGroupedStudies:
    print('Writing ' + DataOutPath)
    df.to_csv(DataOutPath, na_rep="NaN", header=True, index=True)

In [ ]:
print('Done!')